<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparação-dos-Dados" data-toc-modified-id="Preparação-dos-Dados-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparação dos Dados</a></span></li><li><span><a href="#Atentados-pelo-Mundo" data-toc-modified-id="Atentados-pelo-Mundo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Atentados pelo Mundo</a></span></li></ul></div>

Este notebook é uma continuação da implementação _GlobalTerrorism_Parte1_Preparação_, onde foram realizadas um processo extremamente importante de investigação nos dados de modo a mergulhar no contexto e realizar transformações úteis para análises posteriores.

Nessa segunda abordagem, vamos aplicar uma análise exploratória de modo a entender mais sobre os incidentes terroristas registrados ao longo dos anos, buscando por padrões e explicações relacionadas ao contexto. Considerando que os dados já foram filtrados, preparados e transformados de acordo com as tarefas realizadas na _Parte1_, iremos realizar os mesmos procedimentos de forma simplificada para que o objetivo deste notebook seja atingido de forma direta.

O objetivo deste notebook é análisar:

* Quantidade de atentados por ano;
* Atentados que duraram mais de 24h (extended=1);
* Quantidade de atentados por nação, região, cidade ou localização (country_txt, region_txt, city, lat, long);
* Cruzar as informações de localidade com demais atributos (specificity, success, suicide, attacktype1_txt, entre outros);
* Maiores grupos radicais responsáveis por ataques terroristas (gname);
* Locais de ataque com maior número de terroristas (nperps);
* Ferramentas utilizadas em ataque (weaptype1_txt);
* Cruzar informações de ferramentas, locais, grupos radicais por número de mortos por incidente (nkill);
* Cruzar informações de número de terroristas mortos com atributos suicide ou success (nkillter);
* Criar um WordCloud para atributos como summary, corp1, target1 e motive;
* Analisar essa WordCloud agrupada por ano.

# Preparação dos Dados

In [1]:
# Importando biblioteca e lendo arquivo
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

terr = pd.read_csv('/home/paninit/Downloads/datasets/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')
terr.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [2]:
# Filtrando atributos (Parte1 task)
atributos_influentes = ['eventid', 'iyear', 'imonth', 'iday', 'extended', 'country_txt', 'region_txt', 'city', 
                        'latitude', 'longitude', 'specificity', 'summary', 'success', 'suicide', 'attacktype1_txt', 
                        'targtype1_txt', 'copr1', 'target1', 'natlty1_txt', 'gname', 'motive', 'nperps', 
                        'weaptype1_txt', 'nkill', 'nkillter', 'nwound', 'nwoundte', 'ishostkid', 'nhostkid']
terr_data = terr.loc[:, atributos_influentes]
terr_data.head()

,eventid,iyear,imonth,iday,extended,country_txt,region_txt,city,latitude,longitude,...,gname,motive,nperps,weaptype1_txt,nkill,nkillter,nwound,nwoundte,ishostkid,nhostkid
0,197000000001,1970,7,2,0,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,...,MANO-D,NaN,NaN,Unknown,1.0,NaN,0.0,NaN,0.0,NaN
1,197000000002,1970,0,0,0,Mexico,North America,Mexico city,19.371887,-99.086624,...,23rd of September Communist League,NaN,7.0,Unknown,0.0,NaN,0.0,NaN,1.0,1.0
2,197001000001,1970,1,0,0,Philippines,Southeast Asia,Unknown,15.478598,120.599741,...,Unknown,NaN,NaN,Unknown,1.0,NaN,0.0,NaN,0.0,NaN
3,197001000002,1970,1,0,0,Greece,Western Europe,Athens,37.997490,23.762728,...,Unknown,NaN,NaN,Explosives,NaN,NaN,NaN,NaN,0.0,NaN
4,197001000003,1970,1,0,0,Japan,East Asia,Fukouka,33.580412,130.396361,...,Unknown,NaN,NaN,Incendiary,NaN,NaN,NaN,NaN,0.0,NaN


Vamos agora preencher alguns dados nulos de modo a facilitar o entendimento das análises.

In [3]:
terr_data.loc[:, 'city'].fillna('Unknown', inplace=True)
terr_data.loc[:, 'natlty1_txt'].fillna('Unknown', inplace=True)

Criando novas colunas para data e dia da semana do atentado

In [4]:
import numpy as np

terr_data['iday'] = terr_data['iday'].apply(lambda day: day + 1 if day == 0 else day)
terr_data['imonth'] = terr_data['imonth'].apply(lambda month: month + 1 if month == 0 else month)
year = terr_data['iyear'].astype(str)
month = terr_data['imonth'].astype(str)
day = terr_data['iday'].astype(str)
terr_data['event_date'] = year + "/" + month + "/" + day
terr_data['event_date'] = pd.to_datetime(terr_data['event_date'])
terr_data.iloc[:5, np.r_[:4, -1]]

,eventid,iyear,imonth,iday,event_date
0,197000000001,1970,7,2,1970-07-02
1,197000000002,1970,1,1,1970-01-01
2,197001000001,1970,1,1,1970-01-01
3,197001000002,1970,1,1,1970-01-01
4,197001000003,1970,1,1,1970-01-01


In [5]:
terr_data['day_of_week'] = terr_data['event_date'].apply(lambda x: x.dayofweek)
terr_data['day_of_week_name'] = terr_data['event_date'].dt.day_name()
terr_data.head()

,eventid,iyear,imonth,iday,extended,country_txt,region_txt,city,latitude,longitude,...,weaptype1_txt,nkill,nkillter,nwound,nwoundte,ishostkid,nhostkid,event_date,day_of_week,day_of_week_name
0,197000000001,1970,7,2,0,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,...,Unknown,1.0,NaN,0.0,NaN,0.0,NaN,1970-07-02,3,Thursday
1,197000000002,1970,1,1,0,Mexico,North America,Mexico city,19.371887,-99.086624,...,Unknown,0.0,NaN,0.0,NaN,1.0,1.0,1970-01-01,3,Thursday
2,197001000001,1970,1,1,0,Philippines,Southeast Asia,Unknown,15.478598,120.599741,...,Unknown,1.0,NaN,0.0,NaN,0.0,NaN,1970-01-01,3,Thursday
3,197001000002,1970,1,1,0,Greece,Western Europe,Athens,37.997490,23.762728,...,Explosives,NaN,NaN,NaN,NaN,0.0,NaN,1970-01-01,3,Thursday
4,197001000003,1970,1,1,0,Japan,East Asia,Fukouka,33.580412,130.396361,...,Incendiary,NaN,NaN,NaN,NaN,0.0,NaN,1970-01-01,3,Thursday


Alterando alguns países para futuras plotages geográficas

In [6]:
terr_data['country_txt'] = terr_data['country_txt'].apply(lambda x: x.replace('United States', 
                                                                              'United States of America'))

Como informado anteriormente, os procedimentos acima realizados foram extremamente detalhados e explicados na parte 1 do deste projeto de análise dos dados de Terrorismo Global.

# Atentados pelo Mundo

In [11]:
import folium
from folium.plugins import FastMarkerCluster, Fullscreen, MiniMap, HeatMap
import geopandas as gpd
from branca.colormap import LinearColormap
import os
import json

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
world_geo = f'{url}/world-countries.json'
json_data = gpd.read_file(f'{url}/world-countries.json')

In [8]:
def style_function(feature):
    return {
        'fillColor': '#ffaf00',
        'color': 'grey',
        'weight': 1.5,
        'dashArray': '5, 5'
    }

def highlight_function(feature):
    return {
        'fillColor': '#ffaf00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }

In [28]:
country_data = terr_data.groupby(by=['country_txt'], 
                                 as_index=False).count().sort_values(by='eventid', ascending=False).iloc[:, :2]
nkill_data = terr_data.groupby(by=['country_txt'], 
                                 as_index=False).sum().sort_values(by='eventid', 
                                                                   ascending=False).loc[:, ['country_txt', 'nkill']]
temp_global = json_data.merge(country_data, left_on='name', right_on='country_txt', how='left').fillna(0)
global_data = temp_global.merge(nkill_data, left_on='name', right_on='country_txt', how='left').fillna(0)

m = folium.Map(
    location=[0, 0], 
    zoom_start=1.75,
    tiles='openstreetmap'
)

folium.Choropleth(
    geo_data=json_data,
    name='Ataques Terroristas',
    data=country_data,
    columns=['country_txt', 'eventid'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='white',
    nan_fill_opacity=0.9,
    legend_name='Quantidade de Conflitos 1970 - 2017',
    popup_function='Teste'
).add_to(m)

Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)

folium.GeoJson(
    global_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=folium.GeoJsonTooltip(fields=['name', 'eventid', 'nkill'],
                                  aliases=['País:', 'Conflitos:', 'Total de Vítimas'],
                                  labels=True,
                                  sticky=True)
).add_to(m)

m

Inspirações:

https://nbviewer.jupyter.org/gist/jtbaker/57a37a14b90feeab7c67a687c398142c?flush_cache=true

https://github.com/python-visualization/folium/issues/904


In [12]:
heat_data = terr_data.groupby(by=['latitude', 'longitude'], 
                                 as_index=False).count().sort_values(by='eventid', ascending=False).iloc[:, :3]

m = folium.Map(
    location=[33.312805, 44.361488], 
    zoom_start=2.5, 
    tiles='Stamen Toner'
)

HeatMap(
    name='Mapa de Calor',
    data=heat_data,
    radius=10,
    max_zoom=13
).add_to(m)

Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)

#m.add_child(MiniMap())

#folium.LayerControl().add_to(m)

m